In [ ]:
# Microsoft ADAL Library 
# The Python ADAL library will need to be installed.

# Documentation
# https://github.com/AzureAD/azure-activedirectory-library-for-python

# Example
# https://github.com/AzureAD/azure-activedirectory-library-for-python/blob/dev/sample/client_credentials_sample.py

# Note, the Python ADAL library is no longer being maintained and it's documentation suggests to use MSAL. 
# Unfortunately, it doesn't have feature parity with adal at this point and does not support service principal authentication.
import adal

# Provide connection details

In [1]:
# Located in App Registrations from Azure Portal
tenant_id = "<<enter tenant_id>>"

# Authority
authority = "https://login.windows.net/" + tenant_id

# Located in App Registrations from Azure Portal
resource_app_id_url = "https://database.windows.net/"

# In the below example, we're using a Microsoft Spark utility that facilitates acquiring secrets from a configured Key Vault.  

# Service Principal Client ID - Created in App Registrations from Azure Portal
service_principal_id = mssparkutils.credentials.getSecret('azure key vault name','principalClientId')

# Service Principal Secret - Created in App Registrations from Azure Portal
service_principal_secret = mssparkutils.credentials.getSecret('azure key vault name','principalSecret')

# SQL Server URL
servername = "jdbc:sqlserver://azuresqlserver.database.windows.net"
url = servername + ";" + "databaseName=" + dbname + ";"

# Database Name
dbname = "TestDatabase"

# Database Table Name
dbtable = "dbo.TestTable" 

# SQL Authentication User Name
user = "midesa@midesa"

# SQL Authentication Password
password = mssparkutils.credentials.getSecret('azure key vault name','secret name')


StatementMeta(sp03, 66, 1, Finished, Available)

# Active Directory Authentication

In [ ]:
context = adal.AuthenticationContext(authority)
token = context.acquire_token_with_client_credentials(resource_app_id_url, service_principal_id, service_principal_secret)
access_token = token["accessToken"]

jdbc_db = spark.read \
        .format("com.microsoft.sqlserver.jdbc.spark") \
        .option("url", url) \
        .option("dbtable", dbtable) \
        .option("accessToken", access_token) \
        .option("encrypt", "true") \
        .option("hostNameInCertificate", "*.database.windows.net") \
        .load()

In [ ]:
jdbc_df = spark.read \
        .format("com.microsoft.sqlserver.jdbc.spark") \
        .option("url", url) \
        .option("dbtable", dbtable) \
        .option("authentication", "ActiveDirectoryPassword") \
        .option("user", "midesa@microsoft.com") \
        .option("password", "Nostrand@2019") \
        .option("encrypt", "true") \
        .option("hostNameInCertificate", "*.database.windows.net") \
        .load()

# Azure SQL Authentication

In [ ]:
# Write Data
try:
  df.write \
    .format("com.microsoft.sqlserver.jdbc.spark") \
    .mode("overwrite") \
    .option("url", url) \
    .option("dbtable", dbtable) \
    .option("user", user) \
    .option("password", password) \
    .save()
except ValueError as error :
    print("MSSQL Connector write failed", error)

print("MSSQL Connector write(overwrite) succeeded  ")

In [ ]:
# Read Data

print("read data from SQL server table  ")
jdbcDF = spark.read \
        .format("com.microsoft.sqlserver.jdbc.spark") \
        .option("url", url) \
        .option("dbtable", dbtable) \
        .option("user", user) \
        .option("password", password)\
        .load()

jdbcDF.show(5)